# PS3: Neural Networks for Classification and Natural Language Inference

In [1]:
import json
import csv
import os
import glob

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F

from sklearn.metrics import f1_score, precision_score, recall_score

import numpy as np

The purpose of this task is to gain an understanding of training neural networks. Likewise, you will get to learn about the pytorch framework.

## Submission Instructions

After completing the exercises below, generate a pdf of the code **with** outputs. After that create a zip file containing both the completed exercise and the generated PDF. You are **required** to check the PDF to make sure all the code **and** outputs are clearly visible and easy to read. If your code goes off the page, you should reduce the line size. I generally recommend not going over 80 characters.

Finally, name the zip file using a combination of your the assigment and your name, e.g., ps3_rios.zip

## PART I: Data Cleaning (10 points)

Load the "surnames.csv" file to train a LSTM to predict nationality based on surname. You will need to transform the data from a list of strings to a list of indexes. For example, the following data

```
Anthony
John
David
```

should be transformed into a list of lists.

```
[[0, 1, 2, 3, 4, 1, 5],
 [6, 4, 3, 1],
 [7, 8, 9, 10, 11]]
```

Next, you will need zero-pad all examples to be the same size.

```
[[0, 1, 2, 3, 4, 1, 5],
 [6, 4, 3, 1, 0, 0, 0],
 [7, 8, 9, 10, 11, 0, 0]]
```

Finally, everything will be converted into numpy arrays.

In [2]:
char2index = {'<PAD>': 0}
index2char = {0: '<PAD>'}
class2index = {} # stores the class index pairs.
index2class = {}
doc_lengths = [] # Stores the lengths of all docs (train, test and dev)
X_train = [] # stores an 
y_train = [] # stores an index to the correct class
X_dev = []
y_dev = []
X_test = []
y_test = []
X_train_len = [] # Stores the length of each training name
X_test_len = [] # ... length of each test name
X_dev_len = [] # ... length of each dev name

# Write code to load data here.
dataset_filename = 'surnames.csv'
datadir = 'data'
dataset_path = os.path.join(datadir, dataset_filename)
data_names = ['X_train', 'X_dev', 'X_test']

def access_data(path):
    ret = {'X_train':[], 'y_train':[], 'X_dev':[], 'y_dev':[], 'X_test':[], 'y_test':[], 'len':[]}
    with open(path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f,dialect='excel')
        for i, row in enumerate(reader):
#             if i > 5:
#                 break
#             print(row)
            if row[0] == 'train':
                ret['X_train'].append(row[1])
                ret['y_train'].append(row[2])
            if row[0] == 'test':
                ret['X_test'].append(row[1])
                ret['y_test'].append(row[2])
            if row[0] == 'dev':
                ret['X_dev'].append(row[1])
                ret['y_dev'].append(row[2])
            ret['len'].append(len(row[1]))
    return ret
    
def load_data(path):
    
    results = access_data(path)
    class2index = {}
    for class_name in set(results['y_train']+results['y_dev']+results['y_test']):
        class2index[class_name] = len(class2index)
    index2class = {ind:class_name for class_name,ind in class2index.items()}
    return results, class2index, index2class
    
    
def update_mappings(X, x2index, index2x, map_elements=True):
    if map_elements:
        xs = set([element for x in X for element in x])
    else:
        xs = set([x for x in X])
    for x in xs:
#         print("\nx:",x)
#         print(len(x2index))
        x2index[x] = len(x2index)
        index2x[len(index2x)] = x
    
        
def convert_to_index_map(X, x2index, map_element=True):
    index_mappings = []
    for x in X:
        if map_element:
            index_map = [x2index[element] if element in x2index else 0 for element in x]
        else:
            if x in x2index:
                index_map = x2index[x]
            else:
                print('S')
                index_map = 0
        index_mappings.append(index_map)
    return index_mappings
            
    
    
    
data, class2index, index2class = load_data(dataset_path)
X_train, y_train, X_dev, y_dev, X_test, y_test = data['X_train'],data['y_train'],data['X_dev'],data['y_dev'] \
                                                 ,data['X_test'],data['y_test']
doc_lengths = data['len']
d = [X_train, X_dev, X_test]
dl = [X_train_len, X_dev_len, X_test_len]
[dl[i].append(len(x)) for i in range(len(d)) for x in d[i]]
update_mappings(X_train, char2index, index2char)
# print(class2index)
# update_mappings(y_train, class2index, index2class, map_elements=False)
print(char2index)
print(class2index)
print(index2class)

X_train_nums, X_dev_nums = convert_to_index_map(X_train, char2index),convert_to_index_map(X_dev, char2index)
X_test_nums = convert_to_index_map(X_test, char2index)
y_train = convert_to_index_map(y_train, class2index, map_element=False)
y_dev = convert_to_index_map(y_dev, class2index, map_element=False)
y_test = convert_to_index_map(y_test, class2index, map_element=False)

{'<PAD>': 0, '-': 1, 'á': 2, 't': 3, 's': 4, 'K': 5, 'w': 6, 'C': 7, 'ż': 8, 'ê': 9, 'S': 10, 'R': 11, 'o': 12, 'x': 13, 'c': 14, 'Á': 15, 'v': 16, 'l': 17, 'ó': 18, 'q': 19, 'f': 20, 'a': 21, ',': 22, 'Q': 23, 'm': 24, 'ú': 25, 'M': 26, 'ł': 27, 'A': 28, 'B': 29, 'P': 30, 'I': 31, 'H': 32, 'X': 33, 'T': 34, 'k': 35, 'J': 36, 'í': 37, 'h': 38, 'y': 39, 'é': 40, 'g': 41, 'ã': 42, 'õ': 43, 'V': 44, 'd': 45, 'n': 46, 'O': 47, ' ': 48, 'u': 49, 'à': 50, 'F': 51, 'ñ': 52, 'r': 53, 'e': 54, 'Ś': 55, 'D': 56, 'Z': 57, 'ö': 58, 'z': 59, '/': 60, 'i': 61, 'ì': 62, 'U': 63, 'ù': 64, 'j': 65, 'Y': 66, 'G': 67, 'E': 68, 'ü': 69, 'p': 70, 'L': 71, 'ą': 72, 'N': 73, 'è': 74, 'ß': 75, 'ò': 76, 'W': 77, 'b': 78, "'": 79, 'ń': 80, 'ä': 81, '1': 82}
{'french': 0, 'dutch': 1, 'german': 2, 'chinese': 3, 'scottish': 4, 'spanish': 5, 'greek': 6, 'korean': 7, 'vietnamese': 8, 'russian': 9, 'italian': 10, 'portuguese': 11, 'english': 12, 'japanese': 13, 'arabic': 14, 'polish': 15, 'irish': 16, 'czech': 17}
{0

In [3]:
print(len(X_dev))
print(len(X_dev_len))
print(len(doc_lengths))
print(X_train_nums[1])
print(X_train[1])
print(y_train[0])

3060
3060
20074
[30, 53, 61, 35, 21, 59, 14, 38, 61, 35, 12, 16]
Prikazchikov
14


In [4]:
# PADDING

max_seq_len = max(doc_lengths)
len_to_pad = len(max(X_train, key=lambda x: len(x)))
print('longest in training set:', len_to_pad)
X_train_eq_size = []
X_dev_eq_size = []
X_test_eq_size = []

def pad_example(ex, len_to_pad, pad):
    padded = ex[:len_to_pad] +[pad]*(len_to_pad -len(ex))
    return padded
# Write code to append data to code here
for x in X_train_nums:
    X_train_eq_size.append(pad_example(x,len_to_pad, 0))
    
for x in X_dev_nums:
    X_dev_eq_size.append(pad_example(x,len_to_pad, 0))
    
for x in X_test_nums:
    X_test_eq_size.append(pad_example(x,len_to_pad, 0))
    
print(len(X_dev))
X_train = np.array(X_train_eq_size)
X_dev = np.array(X_dev_eq_size)
X_test = np.array(X_test_eq_size)
print(len(X_dev))

y_train = np.array(y_train)
y_dev = np.array(y_dev)
y_test = np.array(y_test)

X_train_len = np.array(X_train_len)
X_dev_len = np.array(X_dev_len)
X_test_len = np.array(X_test_len)

idx = np.argsort(X_dev_len)[::-1]
X_dev = X_dev[idx]
y_dev = y_dev[idx]
X_dev_len = X_dev_len[idx]

idx = np.argsort(X_test_len)[::-1]
X_test = X_test[idx]
y_test = y_test[idx]
X_test_len = X_test_len[idx]

doc_lengths = np.array(doc_lengths)
print(X_train.shape)

longest in training set: 20
3060
3060
(15000, 20)


In [5]:
print(y_train[0])

14


## PART II: Classification (25 points)

In [10]:
class LSTM(nn.Module):
    def __init__(self, nb_layers, word2index, class2index, nb_lstm_units=100,
                 embedding_dim=3, batch_size=3):
        super(LSTM, self).__init__()
        self.vocab = word2index
        self.tags = class2index

        self.nb_layers = nb_layers
        self.nb_lstm_units = nb_lstm_units
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size

        self.nb_tags = len(self.tags)

        # build actual NN
        self.__build_model()

    def __build_model(self):
        nb_vocab_words = len(self.vocab)

        padding_idx = self.vocab['<PAD>']
        self.word_embedding = nn.Embedding(
            num_embeddings=nb_vocab_words,
            embedding_dim=self.embedding_dim,
            padding_idx=padding_idx
        )

        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.nb_lstm_units,
            num_layers=self.nb_layers,
            batch_first=True
        )

        self.hidden_to_tag = nn.Linear(self.nb_lstm_units*self.nb_layers, self.nb_tags)
        
        self.logsoftmax = nn.LogSoftmax(dim=1)
        self.softmax = nn.Softmax(dim=1)
        self.inference = False

    def init_hidden(self, X):
        # Initial ht (hidden state) and ct (context)
        h0 = torch.zeros(self.nb_layers, X.size(0), self.nb_lstm_units).float()
        c0 = torch.zeros(self.nb_layers, X.size(0), self.nb_lstm_units).float()
        return (h0,c0)

    def forward(self, X, X_lengths):
        # reset the LSTM hidden state. Must be done before you run a new batch.
        # Otherwise the LSTM will treat
        # a new batch as a continuation of a sequence
#         print('X start:')
#         print(X)
        self.hidden = self.init_hidden(X)
#         print('hidden')
#         print(self.hidden)
#         print(self.hidden[0].shape)
        batch_size, seq_len = X.shape
        
        # ---------------------
        # 1. embed the input
        # Dim transformation: (batch_size, seq_len) -> (batch_size, seq_len, embedding_dim)
        X = self.word_embedding(X)
#         print('after embedding')
#         print(X)
        # ---------------------
        # 2. Run through RNN
        # TRICK 2 ********************************
        # Dim transformation: (batch_size, seq_len, embedding_dim) -> (batch_size, seq_len, nb_lstm_units)

        # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        X = torch.nn.utils.rnn.pack_padded_sequence(X, X_lengths, batch_first=True)

        # now run through LSTM
        # X contains the padded sequence output and ht contains the final hidden states
        # for example.
        X, (ht, ct) = self.lstm(X, self.hidden)
        
        # Reshape to use the final state from each lstm layer
        out = ht.view(ht.size(1), self.nb_lstm_units*self.nb_layers)

        # pass final states to output layer
        out = self.hidden_to_tag(out)
        
        # Use logsoftmax for training and softmax for testing
        if not self.inference:
            Y_hat = self.logsoftmax(out)
        else:
            Y_hat = self.logsoftmax(out)

        return Y_hat

In [11]:
epochs = 25
batch_size = 8
lstm_unit_size = 512
embedding_size = 512
print_iter = len(y_train)//batch_size//6
# print(print_iter)
num_layers = 1

m = LSTM(num_layers, char2index, class2index, nb_lstm_units = lstm_unit_size,
         embedding_dim = embedding_size, batch_size = batch_size)
xt_len = np.array(X_train_len[:batch_size])
len_idx = xt_len.argsort()[::-1]
# print(xt_len)
# print(len_idx)
bl = torch.tensor(xt_len[len_idx]).long()
# print(bl)
x = torch.tensor(np.array(X_train[:batch_size])[len_idx]).long()
# print(x)
print(m(x,bl))

tensor([[-2.8110, -2.8213, -3.0064, -2.8112, -2.8436, -2.8143, -3.0240, -2.8443,
         -2.8831, -3.0031, -2.9265, -2.8714, -2.9578, -2.7286, -2.9425, -2.9710,
         -2.8801, -2.9446],
        [-2.8444, -3.0102, -2.9584, -2.8008, -2.8262, -2.9741, -2.9563, -2.8374,
         -2.9294, -2.9198, -2.8718, -2.9824, -2.9225, -2.9095, -2.8405, -2.8606,
         -2.7806, -2.8406],
        [-2.8005, -2.9155, -2.7455, -2.9069, -2.9338, -2.8863, -2.9552, -2.8500,
         -2.7989, -2.9505, -2.7973, -3.0162, -2.8965, -3.0008, -2.8984, -2.8070,
         -2.8644, -3.0649],
        [-2.8055, -2.9680, -2.8225, -2.8732, -2.8898, -2.8516, -2.9652, -2.8360,
         -2.8951, -2.8889, -2.8708, -2.9137, -2.9382, -2.8891, -2.9350, -2.7997,
         -2.8550, -3.0657],
        [-2.8616, -2.8654, -2.8868, -2.8150, -2.9549, -2.8741, -2.9295, -2.8625,
         -2.7780, -2.9011, -2.9353, -2.8896, -2.8693, -2.9125, -2.8961, -2.8524,
         -3.0255, -2.9431],
        [-2.7202, -2.9709, -2.9463, -2.9119, -2.88

In [15]:
epochs = 25
batch_size = 64
lstm_unit_size = 512
embedding_size = 512
print_iter = len(y_train)//batch_size//6
print(print_iter)

num_layers = 1

m = LSTM(num_layers, char2index, class2index, nb_lstm_units = lstm_unit_size,
         embedding_dim = embedding_size, batch_size = batch_size)

criterion = nn.NLLLoss(size_average=False)
optim = torch.optim.Adam(m.parameters(), lr=0.0001)

indeces = np.arange(X_train.shape[0])     
print('y_train shape:', y_train.shape)
print('Iterations per epoch:', y_train.shape[0] // batch_size)

for epoch in range(epochs):
    np.random.shuffle(indeces)
    x_train = X_train[indeces]
#     print(x_train.shape)
    y_train2 = y_train[indeces]
#     print('y_train2 shape:', y_train2.shape)
    x_lens = X_train_len[indeces]
        
    current_batch = 0
    for iteration in range(y_train2.shape[0] // batch_size):
        
        batch_lengths = x_lens[current_batch: current_batch + batch_size]
        lengths = np.array(batch_lengths)
        idx = np.argsort(lengths)[::-1]
        batch_lengths = batch_lengths[idx]
        batch_lengths = torch.tensor(batch_lengths).long()
        
        
        batch_x = X_train[current_batch: current_batch + batch_size]
        batch_x = batch_x[idx]
        batch_x = torch.tensor(batch_x).long()
        
        batch_y = y_train2[current_batch: current_batch + batch_size]
        batch_y = batch_y[idx]
#         print(batch_y)
        batch_y = torch.tensor(batch_y).long()
        
        current_batch += batch_size
                        
        optim.zero_grad()
        if len(batch_x) > 0:
#             print(m.inference)
            #batch_pred, batch_y = get_prediction(batch_x, batch_y)
            batch_pred = m(batch_x, batch_lengths)
#             print(batch_size)
#             print(batch_pred)
#             print(batch_y)
            
            loss = criterion(batch_pred, batch_y)
            loss.backward()
            optim.step()

        if iteration % print_iter == 0:
            with torch.no_grad():
                m.train(False)
                m.inference = False
                X_dev = torch.tensor(X_dev).long()
                X_dev_len = torch.tensor(X_dev_len).long()
                batch_pred = np.array(m(X_dev, X_dev_len)).argmax(axis=1)
                batch_y = y_dev
                mic_f1 = f1_score(batch_y, batch_pred, average='micro')
                mac_f1 = f1_score(batch_y, batch_pred, average='macro')
#                 precision = precision_score(batch_y, batch_pred, average='micro')
#                 recall = recall_score(batch_y, batch_pred, average='micro')
                
                print(loss.item(), '\titeraton:', iteration, '\tepoch', epoch, 'micro f1', mic_f1, 'macro f1', mac_f1)
                
                X_test = torch.tensor(X_test).long()
                X_test_len = torch.tensor(X_test_len).long()
                batch_pred = np.array(m(X_test, X_test_len)).argmax(axis=1)
                batch_y = y_test
                f1 = f1_score(batch_y, batch_pred, average='micro')
                mic_f1 = f1_score(batch_y, batch_pred, average='micro')
                mac_f1 = f1_score(batch_y, batch_pred, average='macro')
                print(loss.item(), '\titeraton:', iteration, '\tepoch', epoch, 'micro f1', mic_f1, 'macro f1', mac_f1)
                m.train(True)
                m.inference = False

39
y_train shape: (15000,)
Iterations per epoch: 234


C:\Users\chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


185.33294677734375 	iteraton: 0 	epoch 0 micro f1 0.09967320261437909 macro f1 0.03284978044181922


C:\Users\chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


185.33294677734375 	iteraton: 0 	epoch 0 micro f1 0.10526315789473684 macro f1 0.04944414060811628
117.66925048828125 	iteraton: 39 	epoch 0 micro f1 0.4650326797385621 macro f1 0.03529265873015873
117.66925048828125 	iteraton: 39 	epoch 0 micro f1 0.4642502482621648 macro f1 0.03522851437398741
109.24950408935547 	iteraton: 78 	epoch 0 micro f1 0.46895424836601307 macro f1 0.03834103506682372
109.24950408935547 	iteraton: 78 	epoch 0 micro f1 0.47070506454816285 macro f1 0.039957146590950655
133.31944274902344 	iteraton: 117 	epoch 0 micro f1 0.47320261437908495 macro f1 0.047171766759395634
133.31944274902344 	iteraton: 117 	epoch 0 micro f1 0.46871896722939427 macro f1 0.04810092487769151
117.10587310791016 	iteraton: 156 	epoch 0 micro f1 0.4826797385620915 macro f1 0.0527449255052023
117.10587310791016 	iteraton: 156 	epoch 0 micro f1 0.4786494538232373 macro f1 0.05444032195657714
109.17084503173828 	iteraton: 195 	epoch 0 micro f1 0.49379084967320264 macro f1 0.06228715952668359

KeyboardInterrupt: 

Answer the following questions below:

1. What was the micro and macro F1 on the test and dev sets?
2. Implement a bidirectional LSTM model. You will need to modify the hidden states and self.lstm variables. Does it work better?
3. Experiments with the various hyperparameters (hidden state size, learning rate, etc.). What hyperparemeters result in the best performance?

## PART III: Natural Language Inference (25 points)

Natural language inference is the task of determining whether a "hypothesis" is true (entailment), false (contradiction), or undetermined (neutral) given a "premise"[1, 2]. This task has been known to perform well for zero-shot classification[3].

Example:

| Premise | Label | Hypothesis |
| ------- | ----- | ---------- |
| A man inspects the uniform of a figure in some East Asian country. | contradiction | The man is sleeping. |
| An older and younger man smiling | neutral | Two men are smiling and laughing at the cats playing on the floor. |
| A soccer game with multiple males playing. | entailment | Some men are playing a sport. |

Your task is to load and train a model on the "multinli_1.0_train.jsonl" dataset and evaluate on "multinli_1.0_dev_matched.jsonl" using accuracy.

I am leaving this task relativley open. One solution is to modify the LSTM code above to pass two documents through a LSTM model and return the last hidden state for each. Next, concatenate the two vectors, then pass it through a softmax layer. Finally, train using the same forumlate as Part I.

**NOTE:** You do not need to train until convergence. You can train for only an epoch or 2 max; train less if it takes to long. I simply want to see that it runs and is learning.


[1] Williams, Adina, Nikita Nangia, and Samuel Bowman. "A Broad-Coverage Challenge Corpus for Sentence Understanding through Inference." Proceedings of the 2018 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long Papers). 2018.

[2] Bowman, Samuel R., et al. "A large annotated corpus for learning natural language inference." Proceedings of the 2015 Conference on Empirical Methods in Natural Language Processing. 2015.

[3] Yin, Wenpeng, Jamaal Hay, and Dan Roth. "Benchmarking Zero-shot Text Classification: Datasets, Evaluation and Entailment Approach." Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th International Joint Conference on Natural Language Processing (EMNLP-IJCNLP). 2019.

In [56]:
# COPY AND EDIT CODE HERE

1. Describe your solution.

**ANSWER HERE**

## EXTRA CREDIT 1 (10 points)

Modify the LSTM model to train a language model, then write code to generate new text from the model. Do not forget to mask the loss function when training the language model to handle the different lengths of the sequences. Use the "en-ud-train.upos.tsv" dataset.

Generate 10 examples from your model.

In [ ]:
# COPY AND EDIT CODE HERE